* Все чекпойнты для этой задачи можно скачать по ссылке: https://drive.google.com/drive/folders/13wo8CohBcwmFZHUA06Dl_PISnYfahuXm?usp=sharing
    * Предполагается, что лежат в папке all_data/fs_checkpoints (далее описано, для какого эксперимента используется конкретный чекпойнт)
    * Дополнительно в формате npy с тем же префиксом, что и чекпойнт, лежат сохраненные вектора для некоторых просчетов, можно загрузить их
    *  
    

* Данные для обучения по ссылке (кладутся в основную папку all_data): https://drive.google.com/drive/folders/1oHRzegJm9tId0jliPQPq5G7bYwtwVEK7?usp=sharing

* Для одного из экспериментов требуется предобученная на общей задаче биаффинная модель: сохранить biaffine_checkpoint.pth по пути all_data/general_checkpoints/: https://drive.google.com/drive/folders/1ywaDZ4xX_ahcf0jWMJX69TDMdMhMqtmY?usp=sharing

In [1]:
from scripts.data_utils.dataset_process import get_ind_sequence, dataset
from transformers import get_linear_schedule_with_warmup
from scripts.span_classifier import NNERModel
from scripts.task_trainers import FSTrainer
from torch.utils.data import DataLoader
from torch import cuda, optim
import pandas as pd
import numpy as np
import random
import torch
import re

In [2]:
def set_seed(SEED=42):
    
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    
set_seed()

In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


##### Загрузка данных для обучения и валидации

Перфикс уточняет тип используемых данных
* fs - для начала обучения только по целевым классам или для этапа тестирования
* без префикса - для начала обучения по всем размеченным данным

In [5]:
fs_labels2ids = {'Trash': 0,
                'DISEASE': 1,
                'PENALTY': 2,
                'WORK_OF_ART': 3}
fs_ids2labels = {v:k for k, v in fs_labels2ids.items()}

from scripts.data_utils.label_helper import labels2ids, ids2labels 

train

In [6]:
# тренировочные тексты
ner_ds_path = 'all_data/train_texts.csv'
train_texts = pd.read_csv(ner_ds_path, sep=';')

to_del = [idx for idx, sent in enumerate(train_texts.Contents) if re.match(r'^\s+$', sent)]
train_texts = train_texts.drop(to_del).reset_index()
train_texts = train_texts.drop('index', axis=1)
train_inds = get_ind_sequence(train_texts)

# тренировочные лейблы
ner_ds_path = 'all_data/train_spans.csv'
train_spans = pd.read_csv(ner_ds_path, sep=';')

In [7]:
train_texts_ds = dataset(train_texts, max_len=284)

In [8]:
def get_few_inds(data_texts, data_spans):
    # отбор предложений с нужными few-shot классами для обучения
    address = []
    for row in range(len(data_spans)):
        l = data_spans.Spans.iloc[row].find("'")
        r = data_spans.Spans.iloc[row].rfind("'")
        temp_class = data_spans.Spans.iloc[row][l+1:r]
        if temp_class in fs_labels2ids.keys():
            temp_ad = [data_spans.TextID.iloc[row], data_spans.SentID.iloc[row]]
            if temp_ad not in address:
                address.append(temp_ad)

    tiny_dataset_ids = []
    for row in range(len(data_texts)):
        if [data_texts.TextID.iloc[row], data_texts.SentID.iloc[row]] in address:
            tiny_dataset_ids.append(row)
    return tiny_dataset_ids

In [9]:
# для few-shot только предложения с нужными классами
tiny_dataset_ids = get_few_inds(train_texts, train_spans)
fs_train_texts = train_texts.iloc[tiny_dataset_ids].reset_index()
fs_train_inds = train_inds.iloc[tiny_dataset_ids].reset_index()

fs_train_texts_ds = dataset(fs_train_texts, max_len=284)

test

In [10]:
ner_ds_path = 'all_data/test_texts.csv'

test_texts = pd.read_csv(ner_ds_path, sep=';')
to_del = [idx for idx, sent in enumerate(test_texts.Contents) if re.match(r'^\s+$', sent)]
test_texts = test_texts.drop(to_del).reset_index()
test_texts = test_texts.drop('index', axis=1)
test_inds = get_ind_sequence(test_texts)

ner_ds_path = 'all_data/test_spans.csv'
test_spans = pd.read_csv(ner_ds_path, sep=';')

In [11]:
test_texts_ds = dataset(test_texts, max_len=284)

##### Обучение

Валидация моделей запускается в следующей секции

Используется модель, давшая лучшие результаты на общей задаче - биаффинная, все настройки для нее

Количество эпох зависит от метода обучения
* Если требуется проход по всем размеченным данным (все классы общей задачи), что хватает 20 эпох (use_all_data = True)
* Если проход только по few-shot данным - 200 эпох (use_all_data = False)

Обучение для задачи проводилось в гугл колабе, поэтому для каждого нового этапа подгружалась лучшая модель с предыдущего

In [12]:
# здесь для обучения сначала на полном проходе по размеченным данным
use_all_data = True

In [13]:
TRAIN_BATCH_SIZE = 1
VALID_BATCH_SIZE = 1

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': 1,
                'shuffle': False,
                'num_workers': 0
                }


training_loader = DataLoader(train_texts_ds, **train_params)
testing_loader = DataLoader(test_texts_ds, **test_params)

fs_training_loader = DataLoader(fs_train_texts_ds, **train_params)

При запуске обучения с полных данных, после которых требуется дообучения на few_shot задаче повторный запуск с ячейки ниже

In [14]:
if use_all_data==True:
    use_ids2labels = ids2labels
    use_labels2ids = labels2ids
elif use_all_data==False:
    use_ids2labels = fs_ids2labels
    use_labels2ids = fs_labels2ids

In [15]:
model = NNERModel(num_labels=len(use_ids2labels),
                  device=device,
                  max_seq_len=284,
                  max_span_len=20,
                  num_heads=None,
                  extractor_type='biaffine',
                  mode='extraction',
                  extractor_use_gcn=False)
_ = model.to(device)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
LEARNING_RATE = 1e-05
if use_all_data == True:
    EPOCHS = 20
else:
    EPOCHS = 200


In [17]:
optimizer = optim.Adam(model.parameters(),
                        lr=LEARNING_RATE)

total_steps = len(training_loader) * EPOCHS
warmup_steps = int(total_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps,
                                            num_training_steps = total_steps)

Если с круговой ошибкой обучение должно происходить при полном проходе по данным  со всеми метками классов (**use_all_data==True**), то обучение всегда начинается со свежего сохранения (**from_start = True**), но может быть с предварительным подтягиванием весов модели, обученных для общей задачи (**from_biaf_mode = True**)

Если должны использоваться исключительно few-shot классы, то обучение может быть с самого начала (**from_start = True**), а также с биаффинной моделью (**from_biaf_mode = True**) или c сохранения с полного прохода - тогда **from_start=False**

Для продолжения обучения после полного прохода на few-shot данных требуется указать путь до лучшей сохраненной модели

In [18]:
from_start = True
from_biaf_mode = True

In [19]:
safe_prefix = 'testing_fs_v1'

In [20]:
# проход сначала по полным данным
if use_all_data == True:
    from_start = True

# проход по few-shot примерам
if use_all_data == False:
    if from_start == False:
        from_biaf_mode = False
        check_path = 'new_data/{safe_prefix}_checkpoint.pth'
        checkpoint = torch.load(check_path)
        model.load_state_dict(checkpoint)

In [21]:
if from_biaf_mode == True:
    bf_checkpoint = torch.load(f'all_data/general_checkpoints/biaffine_checkpoint.pth')
    # отбрасываем классифицирующий слой
    drop_layers = list(bf_checkpoint.keys())[-4:]
    for i in drop_layers:
        if i in bf_checkpoint:
            del bf_checkpoint[i]

    # подгрузка весов в новую модель
    n_model = model.state_dict()
    for layer, params in bf_checkpoint.items():
        if layer in n_model.keys():
            n_model[layer] = params
    model.load_state_dict(n_model)

In [22]:
nner_trainer = FSTrainer(model=model,
                       device = device,
                       ids2labels=use_ids2labels,
                       labels2ids=use_labels2ids,
                       optimizer=optimizer,
                       scheduler=scheduler,
                       )

Обозначить название файла для сохранения

In [23]:
safe_prefix = 'testing_fs_v1'

In [ ]:
for epoch in range(EPOCHS):
    print(f'EPOCH {epoch+1}/{EPOCHS}')
    if use_all_data == True:
        epoch_loss = nner_trainer(training_loader, train_spans, train_inds, mode='train')
    elif use_all_data == False:
        epoch_loss = nner_trainer(fs_training_loader, train_spans, fs_train_inds, mode='train')

    if epoch == 0:
        last_best = epoch_loss

    if epoch_loss < last_best:
        last_best = epoch_loss
        check_path = f'new_data/{safe_prefix}_checkpoint.pth'
        torch.save(model.state_dict(),  check_path)

После обучения требуется подгрузить новую модель либо для тестирования (далее), либо для нового прохода по малым данным - для этого нужно вернуться назад и запустить заново код с нужными флагами

In [26]:
# удаление модели для повторной инициализации
del model
# False для продолжения на задаче с малыми классами
use_all_data = False

##### Валидация на тесте с подгружаемой моделью

In [12]:
use_ids2labels = fs_ids2labels
use_labels2ids = fs_labels2ids

In [13]:
model = NNERModel(num_labels=len(use_ids2labels),
                  device=device,
                  max_seq_len=284,
                  max_span_len=20,
                  num_heads=None,
                  extractor_type='biaffine',
                  mode='extraction',
                  extractor_use_gcn=False)
_ = model.to(device)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Предобученные модели для каждого подхода в папке all_data/fs_checkpoints/

Варианты обучения и соответствующее сохранение:

1. Инициализация модели с нуля, проход только по few-shot классам (use_all_data=False) - fs1
2. Инициализация модели с предобученными на общей задаче весами (from_biaf_mode=True) с проходом только по few-shot классам (use_all_data=False) - fs2
3. Инициализация модели с нуля, проход сначала по всем данным со всеми размеченными классами, потом - только по few-shot для донастройки - fs3
4. Инициализация модели с предобученными на общей задаче весами, проход по всем размеченным, потом - только по few-shot - fs4

In [14]:
safe_prefix = 'fs3'

In [15]:
checkpoint = torch.load(f'all_data/fs_checkpoints/{safe_prefix}_checkpoint.pth')
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [16]:
nner_trainer = FSTrainer(model=model,
                       device = device,
                       ids2labels=use_ids2labels,
                       labels2ids=use_labels2ids
                       )

In [17]:
TRAIN_BATCH_SIZE = 1
VALID_BATCH_SIZE = 1

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': 1,
                'shuffle': False,
                'num_workers': 0
                }

fs_training_loader = DataLoader(fs_train_texts_ds, **train_params)
testing_loader = DataLoader(test_texts_ds, **test_params)

Получим вектора для сущностей с помощью предобученной модели

При повторном запуске из-за рандомного выбора отрицательного класса результаты могут отличаться, для сравнения можно использовать сохранения отрезков (сохранения в формате npy)

In [40]:
# загрузить готовое
# path_to_train_spans = f'all_data/fs_checkpoints/{safe_prefix}_spans.npy'
# spans = np.load(path_to_train_spans, allow_pickle=True)

# path_to_train_labels = f'all_data/fs_checkpoints/{safe_prefix}_labels.npy'
# labels = np.load(path_to_train_labels, allow_pickle=True)


In [37]:
spans, labels = nner_trainer(fs_training_loader, train_spans, fs_train_inds, mode='get_spans')

# сохранение
# spans = np.array(spans)
# with open(f'all_data/fs_checkpoints/{safe_prefix}_spans.npy', 'wb') as f:
#         np.save(f, spans)
# labels = np.array(labels)
# with open(f'all_data/fs_checkpoints/{safe_prefix}_labels.npy', 'wb') as f:
#         np.save(f, labels)

57it [00:03, 15.26it/s]


Инициализация KNN, k - число соседей

In [41]:
nner_trainer.init_knn(spans, labels, k=2)

Результирующая таблица

In [42]:
_ = nner_trainer(testing_loader, test_spans, test_inds, mode='test')

935it [05:58,  2.61it/s]


****************SUB NER TYPES********************
DISEASE F1: 0.02467%
DISEASE Recall: 50.87719%
DISEASE Precision: 0.01234%
PENALTY F1: 0.54146%
PENALTY Recall: 44.44444%
PENALTY Precision: 0.27239%
WORK_OF_ART F1: 32.09877%
WORK_OF_ART Recall: 55.91398%
WORK_OF_ART Precision: 22.51082%
Macro F1: 10.88830%
